IMPORT

In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import math


from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.api import OLS
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000


#path = '/Koding_With_Kolesh/challenges/RMB_NOWCAST_APRIL/'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_May_23.csv')
vehicles = pd.read_csv('Naamsa_Vehicle_Sales.csv')
rental = pd.read_csv('PayProp_Rental_Index.csv')
jse = pd.read_csv('jse_indices.csv')
currency = pd.read_csv('currency_data.csv')
expectation = pd.read_excel("Infl_Surv_History2023.xlsx", sheet_name="CPI 5a", header=1)
seed = 3

PIVOTING THE DATA

In [55]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

ADDING JUNE'S DATA MANUALLY FROM https://www.statssa.gov.za/publications/P0141/P0141June2023.pdf

In [56]:
date_str = '2023-06-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_june = [110.9, 104.3, 99.6, 110.4, 118.3, 109.8, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[17, col] = cpi_of_june[i]

ADDIND  ADDITIONAL DATA

* Vehicles https://naamsa.net/wp-content/uploads/2023/07/20230703-Flash-Report-Summary-June-2023.pdf

In [57]:
import datetime


cpi_pivot['year_month'] = pd.to_datetime(cpi_pivot['Month'], format='%Y-%b').dt.strftime('%Y-%m')
start_date = datetime.datetime.strptime("2020-12-31", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2023-05-31", "%Y-%m-%d")

# difference between each date. M means one month end
D = 'M'

date_list = pd.date_range(start_date, end_date, freq=D)[::-1]
vehicles['Date'] = date_list
vehicles['Date'] = pd.to_datetime(vehicles['Date'], format='%Y-%b-%d')
vehicles['year_month'] = pd.to_datetime(vehicles['Date'], format='%Y-%b').dt.strftime('%Y-%m')

cpi_pivot = cpi_pivot.merge(vehicles[['year_month', 'Total_Local Sales', 'Total_Export_Sales']], on='year_month', how='left')
namsa_of_june = [46810, 27296]

for i, col in enumerate(cpi_pivot.columns[-2:]):
    cpi_pivot.at[17, col] = namsa_of_june[i]

* jse_indices

In [58]:
jse['date'] = pd.to_datetime(jse['date'])  # Convert 'Date' column to pandas datetime format
jse.set_index('date', inplace=True)
jse_monthly_avg = jse.resample('M').mean()
cpi_pivot = pd.merge(cpi_pivot,jse_monthly_avg,  right_index=True, left_on='Month', how='inner')
cpi_pivot.drop(['Mid Cap Index', 'Small Cap Index',], axis=1, inplace=True)

* inflation_expectation

In [59]:
cpi_pivot['quarter'] = pd.PeriodIndex(cpi_pivot.Month, freq='Q')
cpi_pivot['quarter'] = cpi_pivot['quarter'].astype(str)
cpi_pivot['quarter'] = cpi_pivot['quarter'].str.replace(r'(\d{4})Q(\d)', r'\1-\2', regex=True)
expectation.rename(columns={"Unnamed: 0" : "names"} , inplace=True)
expectation = expectation.transpose()
expectation.columns = expectation.iloc[0]
expectation = expectation[1:]
expectation.reset_index(inplace=True)
expectation.rename(columns={"index" : "quarter"} , inplace=True)
expectation['quarter'] = expectation['quarter'].str.replace(r'(\d{4})Q(\d)', r'\1-\2', regex=True)
expectation = expectation[expectation['quarter'] >= '2022-1']
expectation.reset_index(drop=True, inplace=True)
cpi_pivot = cpi_pivot.merge(expectation, on='quarter', suffixes=('_cpi', '_expectation'))

ADD THE july ROW

In [60]:
date_str = '2023-07-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING

In [61]:
feats_to_lag = [col for col in cpi_pivot.columns if col not in ['Month', 'year_month', 'quarter']]
for col in feats_to_lag:
  for i in range(2, 10):
    cpi_pivot[f"prev_{i}_month_{col}"]= cpi_pivot[col].shift(i)

HANDLE MISSING DATA

In [62]:
# Extract the columns containing the target names
from sklearn.cluster import KMeans


target_columns = [col for col in cpi_pivot.columns if any(name in col for name in ['Top 40 Index','All-Share Index', 'Large Cap Index', 'Fin 15 Index'])]

# Create a KMeans imputer for each target column separately
for col in target_columns:
    # Initialize the KMeans imputer with the number of clusters you desire
    kmeans_imputer = KMeans(n_clusters=5)  # You can adjust the number of clusters as needed
    
    # Filter non-null values and fit KMeans to them
    non_null_values = cpi_pivot.dropna(subset=[col])
    kmeans_imputer.fit(non_null_values[col].values.reshape(-1, 1))
    
    # Predict cluster centers for non-null values
    non_null_values_clusters = kmeans_imputer.predict(non_null_values[col].values.reshape(-1, 1))
    
    # Reshape cluster centers to 1-dimensional array
    cluster_centers_1d = kmeans_imputer.cluster_centers_.flatten()
    
    # Fill the NaN values in the column with the predicted cluster centers
    cpi_pivot[col].fillna(pd.Series(cluster_centers_1d[non_null_values_clusters], index=non_null_values.index), inplace=True)


In [63]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION

In [64]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-07-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-07-31"]

training_set = train[train['Month']!= '2023-06-30']
validation_set = train[train['Month']== '2023-06-30']

train.shape, test.shape, training_set.shape, validation_set.shape

((17, 210), (1, 210), (16, 210), (1, 210))

MODELING

In [65]:
target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

not_included = ['Top 40 Index', 'All-Share Index',
       'Large Cap Index', 'Fin 15 Index', 'Month', 'year_month','Total_Local Sales', 'Total_Export_Sales', 'quarter', 'Total',	'Analysts',	'Business',	'Labour']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + not_included]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}
bp = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-5, 100)  # Ridge regularization strength
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    copy_X = trial.suggest_categorical('copy_X', [True, False])
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])

    r_model = Ridge(alpha=alpha, fit_intercept=fit_intercept, copy_X=copy_X, max_iter=max_iter, tol=tol, solver=solver,)
    r_model.fit(X_train, y_train[target_col])
    y_pred_colr = r_model.predict(X_val)
    return mean_squared_error(y_val[target_col], y_pred_colr)

#training
for target_col in target_cols:
    #study = optuna.create_study(direction='minimize')
    #study.optimize(objective, n_trials=50)
    #best_params = study.best_params
    
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.1, random_state=seed)
    el_model = LGBMRegressor(learning_rate=0.2, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    #bp[target_col] = best_params
#validation
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)

#scoring

y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T
y_prede = (y_pred + (2 * y_predl) + y_predr) / 4

df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})
dfx = pd.DataFrame({'y_pred': y_predx.flatten(), 'y_val': y_val.values.flatten()})
dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})
dfr = pd.DataFrame({'y_pred': y_predr.flatten(), 'y_val': y_val.values.flatten()})
dfel = pd.DataFrame({'y_pred': y_predel.flatten(), 'y_val': y_val.values.flatten()})
dfe = pd.DataFrame({'y_pred': y_prede.flatten(), 'y_val': y_val.values.flatten()})
#calculate the rmse
rmse = np.sqrt(mean_squared_error(df['y_pred'], df['y_val']))
rmsex = np.sqrt(mean_squared_error(dfx['y_pred'], df['y_val']))
rmsel = np.sqrt(mean_squared_error(dfl['y_pred'], df['y_val']))
rmser = np.sqrt(mean_squared_error(dfr['y_pred'], df['y_val']))
rmseel = np.sqrt(mean_squared_error(dfel['y_pred'], df['y_val']))
ensemble = np.sqrt(mean_squared_error(dfe['y_pred'], df['y_val']))

print(f'RMSE of Liner Regression: {rmse}') 
print(f'RMSE of XGB: {rmsex}') 
print(f'RMSE of Lasso Regression: {rmsel}') 
print(f'RMSE of Riged Regression: {rmser}') 
print(f'RMSE of LGBM Regression: {rmseel}') 
print(f'RMSE of ensemble: {ensemble}') 

RMSE of Liner Regression: 0.4874226308892279
RMSE of XGB: 0.416477194955849
RMSE of Lasso Regression: 1.2612608342290703
RMSE of Riged Regression: 1.3919123468402173
RMSE of LGBM Regression: 4.2115556106530265
RMSE of ensemble: 0.6712232042423427


In [66]:
X_train = train[features]
y_train = train[target_cols]

X_val = test[features]
y_val = test[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.1, random_state=seed)
    el_model = LGBMRegressor(learning_rate=0.2, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    
#pridiction
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)
    
y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T

print(f'pridiction of Liner Regression: {y_pred}') 
print(f'prediction of XGB: {y_predx}') 
print(f'pridiction of Riged Regression: {y_predr}') 
print(f'prediction of Lasso Regression: {y_predl}') 
print(f'pridiction of LGBM Regression: {y_predel}') 

pridiction of Liner Regression: [[111.34412614 104.48518689  99.13317242 110.89351561 118.8441295
  110.59617632 111.15517618 107.67468442 106.0555711  110.26390148
  105.61882307 110.86003135 114.63263356]]
prediction of XGB: [[110.89763  104.29881   99.43773  110.39957  118.29882  109.79882
  110.7975   107.68491  105.39873  109.495094 105.29831  110.19052
  113.730865]]
pridiction of Riged Regression: [[109.67999688 103.61693359  99.29051114 109.67242499 115.882396
  109.62174707 109.8993928  105.91923813 105.25482635 108.69948687
  104.59016464 109.39415613 116.45907255]]
prediction of Lasso Regression: [[110.65109441 104.28659571  99.68823529 109.85358547 119.74854259
  111.99275184 110.1628293  107.96568577 107.32827955 110.00937602
  106.15744294 110.29929447 116.53702306]]
pridiction of LGBM Regression: [[106.09411711 102.34117665  99.68823511 105.55294261 110.74705954
  106.08823574 105.8411762  104.63529385 103.05294037 105.08235348
  102.68235375 106.39411837 110.5       ]]


SUBMITION

In [67]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']
    
    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)
    
    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')
    
    return sub_df[['ID', 'Value']]

In [68]:
y_ins = (y_pred + (2 * y_predl) + y_predr) / 4
sub = prepSub(y_ins, target_cols, test, 'July') 
sub.to_csv('out/ensemble_l2lar_lag210_all.csv', index=False)